<a href="https://colab.research.google.com/github/kattens/Protein-Interaction-with-LLMs/blob/main/REAL_Second_part_model_after_the_protbert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Goal of the project:
the purpose of the model is to be able to efficiently change the number of channels which is a new way of introducing different variables and properties to the context of the protein-protein interaction, as well as being able to change the core model of the architecture to see what is the most suitable core for this approach

###different core models to implement:
1. ESM 2
2. Llama 3
3. ProtBert
4. Bert

###different versions of channels:
1. local inputs with faiss
2. local inputs from datasets based on the UniProt database
3. coordinates of the amino acid's Calpha



In [1]:
%%capture
!pip install --upgrade transformers

In [2]:
import pandas as pd
import numpy as np
import torch

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")


False
CUDA not available


In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#this is the path for the file
csv_file = '/content/drive/MyDrive/pairs.csv'
pairs_df = pd.read_csv(csv_file)

In [ ]:
#theres some values that have ? so we should remove those rows in 'masked_sequence_A', 'masked_sequence_B'
# Remove rows where 'masked_sequence_A' or 'masked_sequence_B' contains '?' anywhere in the sequence
cleaned_df = pairs_df[~(pairs_df['masked_sequence_A'].str.contains('\?', na=False) | pairs_df['masked_sequence_B'].str.contains('\?', na=False))]

# Assign the cleaned DataFrame back to pairs_df
pairs_df = cleaned_df

# Printing the sizes of the original and cleaned dataframes to verify the number of rows removed
print(f"Original DataFrame size: {len(pairs_df)}")
print(f"Cleaned DataFrame size: {len(cleaned_df)}")

Original DataFrame size: 56340
Cleaned DataFrame size: 56340


In [ ]:
# Truncate each specified column to a maximum length of 500 characters
columns = ['masked_sequence_A', 'masked_sequence_B', 'Sequence_A', 'Sequence_B']
for col in columns:
    pairs_df[col] = pairs_df[col].apply(lambda x: x[:500] if len(x) > 500 else x)

# Find the longest string by length
pairs_df['Length'] = pairs_df['masked_sequence_A'].apply(len)
longest_string = pairs_df.loc[pairs_df['Length'].idxmax(), 'masked_sequence_A']
print(len(longest_string))


500


In [ ]:
#the number of protein chains we have
pairs_df.shape[0]

56340

Create dataset class that handles both global sequences and local sequences for protein pairs, and potentially prepares for the inclusion of 3D structural data

#the base Model (without coordinates at this point):

  ### Modeling Interactions:
  The mdel could be trained to recognize which amino acids interact by learning representations of local sequences that highlight these interactions. During training, the MLM objective helps the model learn contextual embeddings that are rich in information about which amino acids tend to be near each other and under what structural contexts they interact.

  ### Attention Mechanism:
   The custom attention mechanism can be used to weigh the importance of different amino acids in the global context when predicting the masked amino acids in the local sequences. This allows your model to focus more on the parts of the global sequences that are relevant to the interactions highlighted by the local sequences.

  ### Utilizing Global Sequences:
  While the local sequences are your primary interest, the global sequences provide the context necessary for your model to understand the broader environment in which the interactions occur. Even during prediction, you should feed the model the global sequences to utilize the learned context.

  #### This modular design not only meets your current requirements but also provides a scalable framework to incorporate additional dimensions of protein sequence data analysis in the future


In [ ]:
!ls /content/drive/MyDrive/Checkpoints

added_tokens.json     generation_config.json   tokenizer_config.json
config.json	      model.safetensors        tokenizer.json
final_checkpoint.pth  special_tokens_map.json  vocab.txt


In [ ]:
#in this block of code we will intialize our core model for the architecture
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

# Define the checkpoint directory and the specific checkpoint file
checkpoint_path = '/content/drive/MyDrive/Checkpoints/final_checkpoint.pth'

# Load the tokenizer first
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Checkpoints')

# Initialize the model from the pre-trained configuration in the Checkpoints directory
model = AutoModelForMaskedLM.from_pretrained('/content/drive/MyDrive/Checkpoints')
model.resize_token_embeddings(len(tokenizer))  # Important if you've added tokens

# Determine the device to use (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to the appropriate device

# Load the checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)  # Ensure checkpoint is loaded to the correct device

# Ensure all keys in the checkpoint can be loaded to the model
missing_keys, unexpected_keys = model.load_state_dict(checkpoint['model_state_dict'], strict=False)
if missing_keys or unexpected_keys:
    print(f"Missing keys in state dict: {missing_keys}")
    print(f"Unexpected keys in state dict: {unexpected_keys}")
else:
    print("Model state loaded successfully.")



"""
An Optimizer and a Learning rate scheduler are being set up and their states are loaded from a checkpoint.
This allows for the continuation of model training with the exact parameters and learning rate adjustments that were in use when the
training was last saved, ensuring a seamless transition and consistency in the training process.
"""

# If you need the optimizer and scheduler states
optimizer = Adam(model.parameters(), lr=checkpoint.get('learning_rate', 0.001))  # Fallback to default if not in checkpoint

# Check if the optimizer state exists in the checkpoint before loading
if 'optimizer_state_dict' in checkpoint:
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
else:
    print("No optimizer state found in checkpoint; starting with a fresh optimizer.")


scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
if 'scheduler_state_dict' in checkpoint:
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
else:
    print("No scheduler state found in checkpoint; using default settings.")

print("Model, tokenizer, optimizer, and scheduler loaded from checkpoint.")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model state loaded successfully.
Model, tokenizer, optimizer, and scheduler loaded from checkpoint.


#the error is in this Dataset class.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class ProteinInteractionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, mask_probability=0.15, modes=None):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability
        self.modes = modes if modes is not None else ['global', 'local', 'coords']
        self.debug = True  # Set this to False to turn off debugging outputs

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        data = {}

        for mode in self.modes:
            sequence = self.get_sequence(row, mode)
            if sequence:
                input_ids, attention_mask = self.tokenize_sequence(sequence)
                data[f'input_ids_{mode}'] = input_ids
                data[f'attention_mask_{mode}'] = attention_mask

                if self.debug:
                    print(f"{mode} mode: input_ids dimension {input_ids.shape}, attention mask dimension {attention_mask.shape}")
                    print(f"{mode} mode: input_ids {input_ids}, attention mask {attention_mask}")

        return data

    def get_sequence(self, row, mode):
        """Helper method to format sequences based on mode."""
        if mode == 'global':
            return f"[CLS] {row['Sequence_A']} [ENTITY1] [SEP] {row['Sequence_B']} [ENTITY2] [SEP]"
        elif mode == 'local':
            return f"[CLS] {row['masked_sequence_A']} [ENTITY1] [SEP] {row['masked_sequence_B']} [ENTITY2] [SEP]"
        elif mode == 'coords':
            return f"[CLS] {row['coords_A']} [ENTITY1] [SEP] {row['coords_B']} [ENTITY2] [SEP]"
        return None

    def tokenize_sequence(self, sequence):
        """Tokenizes the sequence into model-ready input_ids and attention_mask."""
        encoded = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,  # Ensure special tokens are handled correctly
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=1024
        )
        return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)



In [ ]:
# Create a DataFrame from the first row just for debug
debug_df = pairs_df.iloc[[0]]

In [ ]:
debug_df

,pair_id,Protein Name A,Protein Name B,masked_sequence_A,masked_sequence_B,coords_A,coords_B,Embeddings_A,Sequence_A,Embeddings_B,Sequence_B,tokenized_sequence_A,tokenized_sequence_B,tokenized_masked_sequence_A,tokenized_masked_sequence_B,sum_tokenized_sequence_A,sum_tokenized_sequence_B,Length
0,1H0J,1H0J_B,1H0J_C,---------------------------A------------------...,----------------------------T-----------------...,"[(15.18, 27.908, -3.171), (13.094, 28.367, -0....","[(31.71, 11.761, 52.412), (31.514, 9.193, 49.6...","tensor([ 0.1932, -0.0438, 0.7770, 0.2603, -0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"tensor([-1.9606, -0.0789, -1.0516, 1.0233, -1...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"[5, 12, 23, 17, 12, 5, 8, 16, 5, 19, 20, 12, 1...","[5, 12, 23, 17, 12, 5, 8, 16, 5, 19, 20, 12, 1...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[37, 44, 55, 49, 44, 37, 40, 48, 37, 51, 52, 4...","[37, 44, 55, 49, 44, 37, 40, 48, 37, 51, 52, 4...",60


In [ ]:
from torch.utils.data import Dataset

'''
# Initialize the ProtBERT tokenizer -> we dont need it since our tokenizer works perfectly. if we dont have access to the checkpoint we can use this:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('Rostlab/prot_bert')
'''

# Print the length of each sequence column, considering each element as a string
print(f"Sequence_A length: {sum(len(seq) for seq in debug_df['Sequence_A'])}")
print(f"Sequence_B length: {sum(len(seq) for seq in debug_df['Sequence_B'])}")
print(f"Masked_sequence_A length: {sum(len(seq) for seq in debug_df['masked_sequence_A'])}")
print(f"Masked_sequence_B length: {sum(len(seq) for seq in debug_df['masked_sequence_B'])}")

# Sample global sequence preparation
global_seq = "[CLS] " + " [ENTITY1] ".join(debug_df['Sequence_A']) + " [SEP] " + " [ENTITY2] ".join(debug_df['Sequence_B']) + " [SEP]"
local_seq = "[CLS] " + " [ENTITY1] ".join(debug_df['masked_sequence_A']) + " [SEP] " + " [ENTITY2] ".join(debug_df['masked_sequence_B']) + " [SEP]"
coords_seq = "[CLS] " + " [ENTITY1] ".join(debug_df['coords_A']) + " [SEP] " + " [ENTITY2] ".join(debug_df['coords_B']) + " [SEP]"

# Calculate the total length of the sequences
global_seq_length = len(global_seq)
local_seq_length = len(local_seq)
coords_seq_length = len(coords_seq)

# Print the length
print(f"Global sequence length: {global_seq_length}")
print(f"Local sequence length: {local_seq_length}")
print(f"coords sequence length: {coords_seq_length}")

# Assuming ProteinInteractionDataset is properly defined
dataset_global = ProteinInteractionDataset(debug_df, tokenizer, modes=['global'])
dataset_local = ProteinInteractionDataset(debug_df, tokenizer, modes=['local'])
dataset_coords = ProteinInteractionDataset(debug_df, tokenizer, modes=['coords'])

# Fetch data for the first entry in each dataset
data_global = dataset_global[0]
data_local = dataset_local[0]
data_coords = dataset_coords[0]

# Print the data for debugging
print("Global Mode Data:")
for key, value in data_global.items():
    print(f"{key}: {value.shape}")

print("\nLocal Mode Data:")
for key, value in data_local.items():
    print(f"{key}: {value.shape}")

print("\ncoords Mode Data:")
for key, value in data_coords.items():
    print(f"{key}: {value.shape}")


Sequence_A length: 60
Sequence_B length: 60
Masked_sequence_A length: 60
Masked_sequence_B length: 60
Global sequence length: 139
Local sequence length: 139
coords sequence length: 3054
global mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
global mode: input_ids tensor([2, 2, 1,  ..., 0, 0, 0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
local mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
local mode: input_ids tensor([ 2,  2, 32,  ...,  0,  0,  0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
coords mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
coords mode: input_ids tensor([2, 2, 1,  ..., 1, 1, 3]), attention mask tensor([1, 1, 1,  ..., 1, 1, 1])
Global Mode Data:
input_ids_global: torch.Size([1024])
attention_mask_global: torch.Size([1024])

Local Mode Data:
input_ids_local: torch.Size([1024])
attention_mask_local: torch.Size([1024])

coords Mode

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch

def collate_fn(batch):
    # Initialize containers for batch data
    global_input_ids = []
    global_attention_masks = []
    local_input_ids = []
    local_attention_masks = []

    # Collect data for each sample in the batch
    for item in batch:
        global_input_ids.append(item['input_ids_global'])
        global_attention_masks.append(item['attention_mask_global'])
        local_input_ids.append(item['input_ids_local'])
        local_attention_masks.append(item['attention_mask_local'])

    # Pad sequences so they all have the same length within the batch
    global_input_ids = pad_sequence(global_input_ids, batch_first=True, padding_value=0)
    global_attention_masks = pad_sequence(global_attention_masks, batch_first=True, padding_value=0)
    local_input_ids = pad_sequence(local_input_ids, batch_first=True, padding_value=0)
    local_attention_masks = pad_sequence(local_attention_masks, batch_first=True, padding_value=0)

    # Convert lists to tensors
    return {
        'global_input_ids': global_input_ids,
        'global_attention_masks': global_attention_masks,
        'local_input_ids': local_input_ids,
        'local_attention_masks': local_attention_masks
    }


#pretraining the bert model


#main class for training:
  1. Process two sets of sequences (global and local) using BERT to extract contextual embeddings.
  2. Integrate these two sets of embeddings using a custom attention mechanism that focuses on relevant parts of the global features for each part of the local features.
  3. Predict an output (like interaction sites or effects) using the combined features.

#simple class for only 2 channel input of global and local sequneces

In [ ]:
import torch
from torch import nn
from transformers import AutoModel

class SequenceProcessor(nn.Module):
    """A module to process sequences using a pre-trained Transformer model.
    It extracts the last hidden states, which serve as features for further processing."""
    def __init__(self, model):
        super(SequenceProcessor, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        """Perform the forward pass to get sequence features.
        Args:
            input_ids (torch.Tensor): Tensor of input token IDs.
            attention_mask (torch.Tensor, optional): Tensor indicating which tokens should be attended to.
        Returns:
            torch.Tensor: Tensor of last hidden states representing features for each token."""
        return self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

class GuidedAttention(nn.Module):
    """A module that implements a guided attention mechanism to focus on specific parts of a sequence
    based on additional local sequence context."""
    def __init__(self, hidden_size):
        super(GuidedAttention, self).__init__()
        self.key_layer = nn.Linear(hidden_size, hidden_size)
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.value_layer = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        self.context_layer = nn.Linear(hidden_size, hidden_size)

    def forward(self, global_features, local_features):
        """Calculate the attention-driven context vector.
        Args:
            global_features (torch.Tensor): Feature representations of the global sequence.
            local_features (torch.Tensor): Feature representations of the local sequence used for queries.
        Returns:
            torch.Tensor: Processed context vector after applying guided attention."""
        keys = self.key_layer(global_features)
        queries = self.query_layer(local_features)
        values = self.value_layer(global_features)
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (keys.size(-1) ** 0.5)
        attention_weights = self.softmax(attention_scores)
        context = torch.matmul(attention_weights, values)
        processed_context = self.context_layer(context)
        return processed_context

class ProteinInteractionModel(nn.Module):
    """Main model class for predicting protein interaction sites using global and local sequence data.
    This model integrates sequence processing, guided attention, and classification."""
    def __init__(self, model_identifier_or_path, num_labels=21):
        super(ProteinInteractionModel, self).__init__()
        self.base_model = AutoModel.from_pretrained(model_identifier_or_path)
        hidden_size = self.base_model.config.hidden_size

        self.sequence_processor_global = SequenceProcessor(self.base_model)
        self.sequence_processor_local = SequenceProcessor(self.base_model)
        self.guided_attention = GuidedAttention(hidden_size)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids_global, attention_mask_global, input_ids_local, attention_mask_local):
        """Forward computation to predict labels for each sequence position.
        Args:
            input_ids_global (torch.Tensor): Input IDs for the global sequence.
            attention_mask_global (torch.Tensor): Attention mask for the global sequence.
            input_ids_local (torch.Tensor): Input IDs for the local sequence.
            attention_mask_local (torch.Tensor): Attention mask for the local sequence.
        Returns:
            torch.Tensor: Logits for each sequence position indicating predicted label probabilities."""
        global_features = self.sequence_processor_global(input_ids_global, attention_mask_global)
        local_features = self.sequence_processor_local(input_ids_local, attention_mask_local)
        focused_features = self.guided_attention(global_features, local_features)
        logits = self.classifier(focused_features)
        return logits


#more complecated and flexible architecture for adding more channels of input

In [ ]:
'''import torch
from torch import nn
from transformers import AutoModel

class SequenceProcessor(nn.Module):
    """Processes sequences using a pre-trained Transformer model to extract features."""
    def __init__(self, model):
        super(SequenceProcessor, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        """Extracts the last hidden states as features."""
        return self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

class GuidedAttention(nn.Module):
    """Implements attention to focus on relevant parts of a sequence using additional context from other channels."""
    def __init__(self, hidden_size):
        super(GuidedAttention, self).__init__()
        # Define layers for attention mechanism
        self.key_layer = nn.Linear(hidden_size, hidden_size)
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.value_layer = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        self.context_layer = nn.Linear(hidden_size * 2, hidden_size)  # Combines main and context features

    def forward(self, main_features, additional_features):
        """Combines main sequence features with additional features through attention."""
        keys = self.key_layer(main_features)
        queries = self.query_layer(additional_features)
        values = self.value_layer(main_features)

        # Compute attention scores and weights
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (keys.size(-1) ** 0.5)
        attention_weights = self.softmax(attention_scores)

        # Form context vector and combine it with main features
        context = torch.matmul(attention_weights, values)
        combined_features = torch.cat((main_features, context), dim=-1)
        processed_context = self.context_layer(combined_features)
        return processed_context

class FlexibleProteinInteractionModel(nn.Module):
    """Predicts protein interaction sites using multiple sequence data channels with attention integration."""
    def __init__(self, model_identifier_or_path, num_labels):
        super(FlexibleProteinInteractionModel, self).__init__()
        self.base_model = AutoModel.from_pretrained(model_identifier_or_path)
        hidden_size = self.base_model.config.hidden_size

        # Processor for primary sequence features
        self.global_sequence_processor = SequenceProcessor(self.base_model)
        # Processor for secondary features (e.g., local sequences or coordinates)
        self.additional_feature_processor = SequenceProcessor(self.base_model)
        # Guided attention to integrate features from both processors
        self.guided_attention = GuidedAttention(hidden_size)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids_global, attention_mask_global, input_ids_additional, attention_mask_additional):
        """Processes multiple channels of input data and predicts outcomes."""
        global_features = self.global_sequence_processor(input_ids_global, attention_mask_global)
        additional_features = self.additional_feature_processor(input_ids_additional, attention_mask_additional)

        # Integrate global and additional features using guided attention
        integrated_features = self.guided_attention(global_features, additional_features)

        # Classify based on integrated features
        logits = self.classifier(integrated_features)
        return logits
'''

#Training and Validation data creator:
The DataLoader in PyTorch is a powerful utility that simplifies data handling when training neural networks. It's particularly useful when working with large datasets that cannot fit entirely into memory, or when you need sophisticated data loading and batching capabilities.

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into 80% training and 20% testing
train_df, test_df = train_test_split(pairs_df, test_size=0.2, random_state=42)
print(f"Total dataset size: {len(pairs_df)}")
print(f"Training data size: {len(train_df)}")
print(f"Testing data size: {len(test_df)}")

# Assuming ProteinInteractionDataset is correctly implemented to handle DataFrame inputs and a tokenizer
train_dataset = ProteinInteractionDataset(train_df, tokenizer)
test_dataset = ProteinInteractionDataset(test_df, tokenizer)

# Create DataLoader objects for both training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Log the number of batches in each DataLoader
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Each batch has {train_loader.batch_size} samples.")
print(f"Number of batches in test_loader: {len(test_loader)}")
print(f"Each batch has {test_loader.batch_size} samples.")


Total dataset size: 56340
Training data size: 45072
Testing data size: 11268
Number of batches in train_loader: 2817
Each batch has 16 samples.
Number of batches in test_loader: 705
Each batch has 16 samples.


# Model Setup

#Training Loop

In [ ]:
# Extract some batches and check their max token IDs
for batch in train_loader:  # Assuming loader is your DataLoader instance
    max_id_global = batch['input_ids_global'].max()
    max_id_local = batch['input_ids_local'].max()
    max_id_coords = batch['input_ids_coords'].max()  # Assuming you have coords
    print("Max ID Global:", max_id_global.item())
    print("Max ID Local:", max_id_local.item())
    print("Max ID Coords:", max_id_coords.item())
    break  # Remove break to check more batches

# Check against the model's embedding layer size
max_embedding_id = model.base_model.embeddings.word_embeddings.num_embeddings - 1
print("Max embedding index allowed:", max_embedding_id)

# Adjusting the model embeddings to be slightly larger than the max token ID used
required_embedding_size = max(max_id_global.item(), max_id_local.item(), max_id_coords.item()) + 1  # Plus one for safe measure
if required_embedding_size > model.base_model.embeddings.word_embeddings.num_embeddings:
    model.base_model.resize_token_embeddings(required_embedding_size)


global mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
global mode: input_ids tensor([2, 2, 1,  ..., 0, 0, 0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
local mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
local mode: input_ids tensor([ 2,  2, 32,  ...,  0,  0,  0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
coords mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
coords mode: input_ids tensor([2, 2, 1,  ..., 1, 1, 3]), attention mask tensor([1, 1, 1,  ..., 1, 1, 1])
global mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
global mode: input_ids tensor([2, 2, 1,  ..., 0, 0, 0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
local mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
local mode: input_ids tensor([ 2,  2, 32,  ...,  0,  0,  0]), attention mask tensor([1, 1, 1

In [ ]:
import torch
from torch.cuda.amp import GradScaler, autocast

# Assume model, criterion, optimizer, scheduler, train_loader are already defined and configured.

num_epochs = 2  # You can adjust the number of epochs based on your training requirements
scaler = GradScaler()  # Initialize the GradScaler for mixed precision training

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0
    batch_count = 0

    for batch in train_loader:
        optimizer.zero_grad()

        # Move data to the appropriate device (GPU or CPU)
        input_ids_global = batch['input_ids_global'].to(device)
        attention_mask_global = batch['attention_mask_global'].to(device)
        labels = batch['input_ids_local'].to(device)  # Treat local sequences as labels
        attention_mask_local = batch['attention_mask_local'].to(device)  # Might be used depending on model

        # Use automatic mixed precision
        with autocast():
            # Perform a forward pass through the model using global sequences
            outputs = model(input_ids_global, attention_mask_global)

            # Compute loss using the outputs from the model and the local sequences as labels
            loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))

        # Scale the loss and call backward() to create scaled gradients
        scaler.scale(loss).backward()

        # Perform a scaler step. This unscales gradients and calls or skips optimizer.step()
        scaler.step(optimizer)
        scaler.update()

        # Accumulate the loss
        total_loss += loss.item()
        batch_count += 1

    # Clear any unused memory to prevent CUDA out of memory errors
    torch.cuda.empty_cache()

    # Update the learning rate scheduler
    scheduler.step()

    # Calculate average loss over all batches
    average_loss = total_loss / batch_count
    print(f'Epoch: {epoch+1}, Loss: {average_loss:.4f}')

    # Optionally, add code for validation here to evaluate the model


global mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
global mode: input_ids tensor([2, 2, 1,  ..., 0, 0, 0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
local mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
local mode: input_ids tensor([ 2,  2, 32,  ...,  0,  0,  0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
coords mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
coords mode: input_ids tensor([2, 2, 1,  ..., 1, 1, 3]), attention mask tensor([1, 1, 1,  ..., 1, 1, 1])
global mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
global mode: input_ids tensor([2, 2, 1,  ..., 0, 0, 0]), attention mask tensor([1, 1, 1,  ..., 0, 0, 0])
local mode: input_ids dimension torch.Size([1024]), attention mask dimension torch.Size([1024])
local mode: input_ids tensor([ 2,  2, 32,  ...,  0,  0,  0]), attention mask tensor([1, 1, 1

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 